# 从线性回归到神经网络之逻辑回归篇
## 名为回归的分类算法

In [131]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('data/heart(1).csv')

In [3]:
data.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [8]:
X = data.iloc[:,:-1]
y = data['target'].values
y = y.reshape(-1,1)
print(f'特征集形状: {X.shape}')
print(f'标签集形状: {y.shape}')

特征集形状: (303, 13)
标签集形状: (303, 1)


In [14]:
X.T.shape[0]

13

机器学习的分类过程,也就是确定某一个事物,隶属于某一个类别的可能性大小的过程

1. 找到sigmoid函数: $$g(z) = \frac{1}{1+e^{-z}}$$

2. 找到线性回归模型店中间值: $$z(x)=w_1 x_1+w_2 x_2+\ldots+w_{n-1} x_{n-1}+w_n x_n+b=W^TX$$

3. 将1和2组合起来,将函数表示为假设函数形式:
$$g(x) = \frac{1}{1+e^{-(w_1 x_1+w_2 x_2+\ldots+w_{n-1} x_{n-1}+w_n x_n+b)}}$$


最终得到逻辑回归的假设函数
$$h(x) = \frac{1}{1+e^{-(W^TX)}}$$

In [53]:
#求预测值函数
def sigmoid(z):
    y_hat = 1 / (1 + np.exp(-z))
    return y_hat

In [54]:
#求损失值函数
def loss(X,y,w,b):
    y_hat = sigmoid(np.dot(X,w)+ b)
    loss = -(y*np.log(y_hat) + (1-y) * np.log(1 - y_hat))
    loss = np.sum(loss) / len(X)
    return loss

### 逻辑回归的梯度下降
$$\text{梯度} = h^{\prime}(x)=\frac{\partial}{\partial w} L(w, b)=\frac{\partial}{\partial w}\left\{-\frac{1}{N} \sum_{(x, y) \in D}[y * \log (h(x))+(1-y) * \log (1-h(x))]\right\}$$


$$\text{梯度} = \frac{1}{N} \sum_{i=1}^N\left(y^{(i)}-h\left(x^{(i)}\right)\right) \cdot x^{(i)}$$

$$\frac{1}{N} \sum_{i=1}^N\left(y^{(i)}-h\left(x^{(i)}\right)\right) \cdot x^{(i)}$$


$$w=w-\frac{\alpha}{N} \sum_{j=1}^N\left(y^{(i)}-\left(w \cdot x^{(i)}\right)\right) \cdot x^{(i)}$$

In [49]:
# 逻辑回归的梯度下降 ：
def gradient_descent(X, y, w, b, lr, iters) : # 定义逻辑回归梯度下降函数
    l_history = np.zeros(iters) # 初始化记录梯度下降过程中误差值 ( 损失 ) 的数组
    w_history = np.zeros((iters, w.shape[0], w.shape[1])) # 初始化记录梯度下降过程中权重的数组
    b_history = np.zeros(iters) # 初始化记录梯度下降过程中偏置的数组 
    for i in range(iters): # 进行机器训练的迭代
        y_hat = sigmoid(np.dot(X, w) + b) #Sigmoid 逻辑函数 + 线性函数 (wX+b) 得到 y'
        # loss1 = -(y*np.log(y_hat) + (1-y)*np.log(1-y_hat)) # 计算损失
        derivative_w = np.dot(X.T, (y_hat-y))/X.shape[0] # 给权重向量求导
        derivative_b = np.sum(y_hat-y)/X.shape[0] # 给偏置求导
        w = w - lr * derivative_w # 更新权重向量 , lr 即学习速率 alpha
        b = b - lr * derivative_b # 更新偏置 , lr 即学习速率 alpha
        l_history[i] = loss(X, y, w, b) # 梯度下降过程中的损失
        print (" 轮次 ", i+1 , " 当前轮训练集损失 ：", l_history[i]) 
        w_history[i] = w # 梯度下降过程中权重的历史记录，请注意 w_history 和 w 的形状
        b_history[i] = b # 梯度下降过程中偏置的历史记录
    return l_history, w_history, b_history

##  通过逻辑回归解决二元分类问题

In [123]:
plt.rcParams['font.sans-serif'] = 'Times New Roman'
import matplotx
#年龄/最大心率,是否患病之间的关系
with plt.style.context(matplotx.styles.pitaya_smoothie['light']):
    plt.figure(figsize = (12,6),dpi = 100)
    plt.scatter(x = data[data['target'] == 1]['age'],y = data[data['target'] == 1]['thalach'],color = 'red',marker = 'o',s = 40,label = 'Disease')
    plt.scatter(x = data[data['target'] == 0]['age'],y = data[data['target'] == 0]['thalach'],color = 'blue',marker = 'o',s = 40,label = 'Normal')
    plt.legend()
    plt.xlabel('Age')
    plt.ylabel('Thalach')
plt.savefig('result/年龄最大心率是否患病之间的关系.png')

In [21]:
X = data.iloc[:,:-1]
y = data['target']

In [27]:
y = y.values.reshape(-1,1)

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1210)

In [29]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

In [83]:
#预测函数的定义
def predict(X,w,b):
    y_hat = sigmoid(np.dot(X,w)+ b)
    y_pred = np.zeros((y_hat.shape[0],1))
    for i in range(y_hat.shape[0]):
        if y_hat[i,0] <0.5:
            y_pred[i,0] = 0
        else :
            y_pred[i,0] = 1
    return y_pred

In [96]:
def logistic_regression(X,y,w,b,lr,iters):
    l_history,w_history,b_history = gradient_descent(X,y,w,b,lr,iters)
    print(f'训练最终损失:{l_history[-1]}')
    w = w_history[-1]
    b = b_history[-1]
    y_pred = predict(X,w,b)
    trainning_acc = 100 - np.mean(np.abs(y_pred - y)) * 100
    print(f"逻辑回归训练的准确率为:{trainning_acc: .2f}%")
    return l_history,w_history,b_history

In [97]:
# 初始化参数
dimension = X.shape[1]
weight = np.full((dimension,1),0.1)
bias = 0
# 初始化超参数
alpha = 0.5 # 学习速率
iterations = 1000   # 迭代次数

In [98]:
loss_history, weight_history, bias_history = \
    logistic_regression(X_train,y_train,weight,bias,alpha,iterations)

 轮次  1  当前轮训练集损失 ： 0.6854176219094903
 轮次  2  当前轮训练集损失 ： 0.6692134316680488
 轮次  3  当前轮训练集损失 ： 0.6557837317504402
 轮次  4  当前轮训练集损失 ： 0.6437244803630164
 轮次  5  当前轮训练集损失 ： 0.6325832860506098
 轮次  6  当前轮训练集损失 ： 0.6221924911818881
 轮次  7  当前轮训练集损失 ： 0.612468713010015
 轮次  8  当前轮训练集损失 ： 0.6033549049403493
 轮次  9  当前轮训练集损失 ： 0.5948038179958415
 轮次  10  当前轮训练集损失 ： 0.5867732303553008
 轮次  11  当前轮训练集损失 ： 0.5792244891140443
 轮次  12  当前轮训练集损失 ： 0.572121978803147
 轮次  13  当前轮训练集损失 ： 0.5654328476858228
 轮次  14  当前轮训练集损失 ： 0.5591268058524832
 轮次  15  当前轮训练集损失 ： 0.5531759448189439
 轮次  16  当前轮训练集损失 ： 0.5475545662318351
 轮次  17  当前轮训练集损失 ： 0.5422390175875262
 轮次  18  当前轮训练集损失 ： 0.5372075353870139
 轮次  19  当前轮训练集损失 ： 0.5324400965071601
 轮次  20  当前轮训练集损失 ： 0.527918278376447
 轮次  21  当前轮训练集损失 ： 0.5236251282578686
 轮次  22  当前轮训练集损失 ： 0.5195450416861217
 轮次  23  当前轮训练集损失 ： 0.5156636499070419
 轮次  24  当前轮训练集损失 ： 0.5119677160225001
 轮次  25  当前轮训练集损失 ： 0.5084450394450776
 轮次  26  当前轮训练集损失 ： 0.505084368204242

In [103]:
data = pd.read_csv('./data/heart(1).csv')
a = pd.get_dummies(data['cp'],prefix='cp')
b = pd.get_dummies(data['thal'], prefix='thal')
c = pd.get_dummies(data['slope'], prefix='slope')
frames = [a,b,c,data]
data = pd.concat(frames,axis=1)
data = data.drop(columns=['cp','thal','slope'])
data.head()

,cp_0,cp_1,cp_2,cp_3,thal_0,thal_1,thal_2,thal_3,slope_0,slope_1,...,sex,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,ca,target
0,False,False,False,True,False,True,False,False,True,False,...,1,145,233,1,0,150,0,2.3,0,1
1,False,False,True,False,False,False,True,False,True,False,...,1,130,250,0,1,187,0,3.5,0,1
2,False,True,False,False,False,False,True,False,False,False,...,0,130,204,0,0,172,0,1.4,0,1
3,False,True,False,False,False,False,True,False,False,False,...,1,120,236,0,1,178,0,0.8,0,1
4,True,False,False,False,False,False,True,False,False,False,...,0,120,354,0,1,163,1,0.6,0,1


In [107]:
data.shape

(303, 22)

In [108]:
X = data.iloc[:,:-1]
y = data['target'].values
y = y.reshape(-1,1)
print(f'特征集形状: {X.shape}')
print(f'标签集形状: {y.shape}')
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

特征集形状: (303, 21)
标签集形状: (303, 1)


In [109]:
# 初始化参数
dimension = X.shape[1]
weight = np.full((dimension,1),0.1)
bias = 0
# 初始化超参数
alpha = 1 # 学习速率
iterations = 3000   # 迭代次数

In [118]:
loss_history, weight_history, bias_history = \
    logistic_regression(X_train, y_train, weight, bias, alpha, iterations)

 轮次  1  当前轮训练集损失 ： 0.6222467622787473
 轮次  2  当前轮训练集损失 ： 0.5679450127855463
 轮次  3  当前轮训练集损失 ： 0.5308013487322748
 轮次  4  当前轮训练集损失 ： 0.5043965878693714
 轮次  5  当前轮训练集损失 ： 0.4850272706593694
 轮次  6  当前轮训练集损失 ： 0.47040417019318226
 轮次  7  当前轮训练集损失 ： 0.4590781878636297
 轮次  8  当前轮训练集损失 ： 0.4501057746298244
 轮次  9  当前轮训练集损失 ： 0.4428548588411144
 轮次  10  当前轮训练集损失 ： 0.4368905967069161
 轮次  11  当前轮训练集损失 ： 0.4319064979082156
 轮次  12  当前轮训练集损失 ： 0.42768179348834606
 轮次  13  当前轮训练集损失 ： 0.42405434407808884
 轮次  14  当前轮训练集损失 ： 0.4209029883377424
 轮次  15  当前轮训练集损失 ： 0.41813577235349875
 轮次  16  当前轮训练集损失 ： 0.41568193167574335
 轮次  17  当前轮训练集损失 ： 0.41348632224634146
 轮次  18  当前轮训练集损失 ： 0.41150548295290085
 轮次  19  当前轮训练集损失 ： 0.4097048062978073
 轮次  20  当前轮训练集损失 ： 0.40805647500660996
 轮次  21  当前轮训练集损失 ： 0.4065379367021377
 轮次  22  当前轮训练集损失 ： 0.40513076223937955
 轮次  23  当前轮训练集损失 ： 0.40381978138700875
 轮次  24  当前轮训练集损失 ： 0.4025924215590717
 轮次  25  当前轮训练集损失 ： 0.40143819695604727
 轮次  26  当前轮训练集损失 ： 0.4

In [119]:
loss_history_test = np.zeros(iterations)
for i in range(iterations):
    loss_history_test[i] = loss(X_test,y_test,weight_history[i],bias_history[i])

In [129]:
plt.figure(figsize=(12,6),dpi = 100)
plt.plot(np.arange(0,iterations,1),loss_history_test,c = 'blue',label = 'LossTest')
plt.plot(np.arange(0,iterations,1),loss_history,c = 'red',linestyle = 'dotted',label = 'LossTrain')
plt.legend()
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.savefig('result/哑变量-心脏病测试集的损失值和训练集损失值.png')

In [122]:
#用sklearn来实现 仅需下面几行代码 0.0
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

D:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8852459016393442